In [75]:
import pandas as pd
import numpy as np

# 데이터 로드
orders = pd.read_csv('Olist_DataSet/olist_orders_dataset.csv')

# 날짜 컬럼 변환
date_cols = ['order_purchase_timestamp', 'order_approved_at', 
             'order_delivered_carrier_date', 'order_delivered_customer_date',
             'order_estimated_delivery_date']

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col])

print("데이터 기본 정보 (필터링 전):")
print(f"전체 주문 수: {len(orders):,}")

# ==========================================
# 불완전한 기간 데이터 제외
# - 2016년 9월 ~ 12월
# - 2018년 9월 ~ 11월
# ==========================================
orders['year_month'] = orders['order_purchase_timestamp'].dt.to_period('M')

# 제외할 기간 정의
exclude_periods = (
    # 2016년 9월 ~ 12월
    ((orders['order_purchase_timestamp'].dt.year == 2016) & 
     (orders['order_purchase_timestamp'].dt.month >= 9)) |
    # 2018년 9월 ~ 11월
    ((orders['order_purchase_timestamp'].dt.year == 2018) & 
     (orders['order_purchase_timestamp'].dt.month >= 9) &
     (orders['order_purchase_timestamp'].dt.month <= 11))
)

# 제외된 데이터 건수 확인
excluded_orders = orders[exclude_periods]
print(f"\n제외된 주문 수: {len(excluded_orders):,}")

# 필터링 적용
orders = orders[~exclude_periods].copy()
orders = orders.drop(columns=['year_month'])

print(f"\n필터링 후 주문 수: {len(orders):,}")
print(f"\n날짜 컬럼별 결측치:")
print(orders[date_cols].isnull().sum())

데이터 기본 정보 (필터링 전):
전체 주문 수: 99,441

제외된 주문 수: 349

필터링 후 주문 수: 99,092

날짜 컬럼별 결측치:
order_purchase_timestamp            0
order_approved_at                 135
order_delivered_carrier_date     1716
order_delivered_customer_date    2888
order_estimated_delivery_date       0
dtype: int64


In [76]:
# 배송 관련 이상 데이터 탐지
anomalies = {}

# 1. 운송업체 인수일 > 고객 배송 완료일 (논리적으로 불가능)
# 운송업체가 물건을 받은 날이 고객이 받은 날보다 늦음
anomaly_1 = orders[
    (orders['order_delivered_carrier_date'].notna()) & 
    (orders['order_delivered_customer_date'].notna()) &
    (orders['order_delivered_carrier_date'] > orders['order_delivered_customer_date'])
]
anomalies['1. 운송업체 인수일 > 고객 배송일'] = anomaly_1

print("=" * 60)
print("1. 운송업체 인수일(carrier) > 고객 배송일(customer)")
print("   -> 운송업체가 물건을 받기도 전에 고객이 받음?!")
print("=" * 60)
print(f"이상 건수: {len(anomaly_1):,}건")
if len(anomaly_1) > 0:
    print("\n샘플 데이터:")
    display(anomaly_1[['order_id', 'order_status', 'order_delivered_carrier_date', 
                       'order_delivered_customer_date']].head(10))


1. 운송업체 인수일(carrier) > 고객 배송일(customer)
   -> 운송업체가 물건을 받기도 전에 고객이 받음?!
이상 건수: 19건

샘플 데이터:


,order_id,order_status,order_delivered_carrier_date,order_delivered_customer_date
6437,a1abeb653a4d4cd1e142ccb8c82cd069,delivered,2017-07-28 16:57:58,2017-07-25 19:32:56
9553,383aa8b2724fe452d9ccd9934a8c628b,delivered,2017-07-07 17:22:41,2017-07-06 14:27:51
13487,cb1134f9010d242e9515ad1c78ec0c39,delivered,2017-07-20 19:22:02,2017-07-19 14:13:28
14474,dceb62e8fa94b46006c9554fed743df0,delivered,2017-08-01 18:23:30,2017-07-26 18:09:10
19268,5f9d46795c3126674e52becb3a1a517f,delivered,2017-07-20 23:03:42,2017-07-20 18:52:41
22520,b27af682321527a6349f1761eb3f360c,delivered,2017-06-27 14:51:54,2017-06-26 15:45:35
25393,1cc3ae63caffff2d6c3ee3e78e074acf,delivered,2017-08-10 18:28:56,2017-08-10 18:05:38
25646,e37f11cae9985ca58f0b56f268720537,delivered,2017-08-01 18:17:47,2017-07-31 17:49:56
27470,fa3e37584f4fdb1ded0e0de700dfcb4e,delivered,2017-08-09 18:18:43,2017-08-01 21:13:01
34939,c1e2bf2b7dd3309f2f5356c6b63968fa,delivered,2017-03-02 17:34:26,2017-02-14 15:15:57


In [77]:
# 2. 주문 승인일 > 운송업체 인수일 (논리적으로 불가능)
# 주문이 승인되기 전에 운송업체가 물건을 받음
anomaly_2 = orders[
    (orders['order_approved_at'].notna()) & 
    (orders['order_delivered_carrier_date'].notna()) &
    (orders['order_approved_at'] > orders['order_delivered_carrier_date'])
]
anomalies['2. 주문 승인일 > 운송업체 인수일'] = anomaly_2

print("=" * 60)
print("2. 주문 승인일(approved) > 운송업체 인수일(carrier)")
print("   -> 주문 승인 전에 운송업체가 물건을 받음?!")
print("=" * 60)
print(f"이상 건수: {len(anomaly_2):,}건")
if len(anomaly_2) > 0:
    print("\n샘플 데이터:")
    display(anomaly_2[['order_id', 'order_status', 'order_approved_at', 
                       'order_delivered_carrier_date']].head(10))


2. 주문 승인일(approved) > 운송업체 인수일(carrier)
   -> 주문 승인 전에 운송업체가 물건을 받음?!
이상 건수: 1,359건

샘플 데이터:


,order_id,order_status,order_approved_at,order_delivered_carrier_date
15,dcb36b511fcac050b97cd5c05de84dc3,delivered,2018-06-12 23:31:02,2018-06-11 14:54:00
64,688052146432ef8253587b930b01a06d,delivered,2018-04-24 18:25:22,2018-04-23 19:19:14
199,58d4c4747ee059eeeb865b349b41f53a,delivered,2018-07-26 23:31:53,2018-07-24 12:57:00
210,412fccb2b44a99b36714bca3fef8ad7b,delivered,2018-07-23 12:31:53,2018-07-23 12:24:00
415,56a4ac10a4a8f2ba7693523bb439eede,delivered,2018-07-27 23:31:09,2018-07-24 14:03:00
481,32e4fa9bb468884309b58b9348de70c3,delivered,2018-07-05 16:33:06,2018-07-05 14:50:00
483,4df92d82d79c3b52c7138679fa9b07fc,delivered,2018-07-29 23:30:52,2018-07-26 14:46:00
585,16e38caa92e342c7780f68832f832d4d,delivered,2018-05-07 16:52:39,2018-05-07 15:09:00
615,b9afddbdcfadc9a87b41a83271c3e888,delivered,2018-08-16 14:05:13,2018-08-16 13:27:00
817,6051e6d3da9a50b7325cbe9c81025062,delivered,2018-07-05 16:31:26,2018-07-04 12:14:00


In [78]:
# 3. 주문일 > 주문 승인일 (논리적으로 불가능)
# 주문하기 전에 승인됨
anomaly_3 = orders[
    (orders['order_purchase_timestamp'].notna()) & 
    (orders['order_approved_at'].notna()) &
    (orders['order_purchase_timestamp'] > orders['order_approved_at'])
]
anomalies['3. 주문일 > 주문 승인일'] = anomaly_3

print("=" * 60)
print("3. 주문일(purchase) > 주문 승인일(approved)")
print("   -> 주문하기 전에 승인됨?!")
print("=" * 60)
print(f"이상 건수: {len(anomaly_3):,}건")
if len(anomaly_3) > 0:
    print("\n샘플 데이터:")
    display(anomaly_3[['order_id', 'order_status', 'order_purchase_timestamp', 
                       'order_approved_at']].head(10))


3. 주문일(purchase) > 주문 승인일(approved)
   -> 주문하기 전에 승인됨?!
이상 건수: 0건


In [79]:
# 4. 주문일 > 고객 배송일 (논리적으로 불가능)
# 주문하기 전에 배송 완료됨
anomaly_4 = orders[
    (orders['order_purchase_timestamp'].notna()) & 
    (orders['order_delivered_customer_date'].notna()) &
    (orders['order_purchase_timestamp'] > orders['order_delivered_customer_date'])
]
anomalies['4. 주문일 > 고객 배송일'] = anomaly_4

print("=" * 60)
print("4. 주문일(purchase) > 고객 배송일(customer)")
print("   -> 주문하기 전에 배송 완료됨?!")
print("=" * 60)
print(f"이상 건수: {len(anomaly_4):,}건")
if len(anomaly_4) > 0:
    print("\n샘플 데이터:")
    display(anomaly_4[['order_id', 'order_status', 'order_purchase_timestamp', 
                       'order_delivered_customer_date']].head(10))


4. 주문일(purchase) > 고객 배송일(customer)
   -> 주문하기 전에 배송 완료됨?!
이상 건수: 0건


In [80]:
# 5. 배송완료(delivered) 상태인데 배송 날짜가 없는 경우
anomaly_5 = orders[
    (orders['order_status'] == 'delivered') & 
    (orders['order_delivered_customer_date'].isna())
]
anomalies['5. 배송완료 상태인데 배송일 없음'] = anomaly_5

print("=" * 60)
print("5. 배송완료(delivered) 상태인데 고객 배송일(customer)이 없음")
print("   -> 상태는 배송완료인데 실제 배송일 기록이 없음?!")
print("=" * 60)
print(f"이상 건수: {len(anomaly_5):,}건")
if len(anomaly_5) > 0:
    print("\n샘플 데이터:")
    display(anomaly_5[['order_id', 'order_status', 'order_delivered_carrier_date',
                       'order_delivered_customer_date']].head(10))


5. 배송완료(delivered) 상태인데 고객 배송일(customer)이 없음
   -> 상태는 배송완료인데 실제 배송일 기록이 없음?!
이상 건수: 8건

샘플 데이터:


,order_id,order_status,order_delivered_carrier_date,order_delivered_customer_date
3002,2d1e2d5bf4dc7227b3bfebb81328c15f,delivered,2017-11-30 18:12:23,NaT
20618,f5dd62b788049ad9fc0526e3ad11a097,delivered,2018-06-25 08:05:00,NaT
43834,2ebdfc4f15f23b91474edf87475f108e,delivered,2018-07-03 13:57:00,NaT
79263,e69f75a717d64fc5ecdfae42b2e8e086,delivered,2018-07-03 13:57:00,NaT
82868,0d3268bad9b086af767785e3f0fc0133,delivered,2018-07-03 09:28:00,NaT
92643,2d858f451373b04fb5c984a1cc2defaf,delivered,NaT,NaT
97647,ab7c89dc1bf4a1ead9d6ec1ec8968a84,delivered,2018-06-12 14:10:00,NaT
98038,20edc82cf5400ce95e1afacc25798b31,delivered,2018-07-03 19:26:00,NaT


In [81]:
# 6. 예상 배송일 < 주문일 (논리적으로 불가능)
# 예상 배송일이 주문일보다 먼저
anomaly_6 = orders[
    (orders['order_purchase_timestamp'].notna()) & 
    (orders['order_estimated_delivery_date'].notna()) &
    (orders['order_estimated_delivery_date'] < orders['order_purchase_timestamp'])
]
anomalies['6. 예상 배송일 < 주문일'] = anomaly_6

print("=" * 60)
print("6. 예상 배송일(estimated) < 주문일(purchase)")
print("   -> 예상 배송일이 주문일보다 먼저?!")
print("=" * 60)
print(f"이상 건수: {len(anomaly_6):,}건")
if len(anomaly_6) > 0:
    print("\n샘플 데이터:")
    display(anomaly_6[['order_id', 'order_status', 'order_purchase_timestamp', 
                       'order_estimated_delivery_date']].head(10))


6. 예상 배송일(estimated) < 주문일(purchase)
   -> 예상 배송일이 주문일보다 먼저?!
이상 건수: 0건


In [82]:
# 7. 배송 시간 계산 (운송업체 인수 → 고객 배송)
# 배송 시간이 음수인 경우 (같은 날 또는 이전)
orders['delivery_time_days'] = (
    orders['order_delivered_customer_date'] - orders['order_delivered_carrier_date']
).dt.total_seconds() / (24 * 60 * 60)

# 음수인 경우
anomaly_7 = orders[
    (orders['delivery_time_days'].notna()) & 
    (orders['delivery_time_days'] < 0)
]
anomalies['7. 배송 시간 0일 이하'] = anomaly_7

print("=" * 60)
print("7. 배송 시간(carrier→customer)이 0일 이하")
print("   -> 운송업체가 받은 당일 또는 그 전에 고객에게 배송됨?!")
print("=" * 60)
print(f"이상 건수: {len(anomaly_7):,}건")
if len(anomaly_7) > 0:
    print("\n샘플 데이터:")
    display(anomaly_7[['order_id', 'order_status', 'order_delivered_carrier_date', 
                       'order_delivered_customer_date', 'delivery_time_days']].head(10))


7. 배송 시간(carrier→customer)이 0일 이하
   -> 운송업체가 받은 당일 또는 그 전에 고객에게 배송됨?!
이상 건수: 19건

샘플 데이터:


,order_id,order_status,order_delivered_carrier_date,order_delivered_customer_date,delivery_time_days
6437,a1abeb653a4d4cd1e142ccb8c82cd069,delivered,2017-07-28 16:57:58,2017-07-25 19:32:56,-2.892384
9553,383aa8b2724fe452d9ccd9934a8c628b,delivered,2017-07-07 17:22:41,2017-07-06 14:27:51,-1.121412
13487,cb1134f9010d242e9515ad1c78ec0c39,delivered,2017-07-20 19:22:02,2017-07-19 14:13:28,-1.214282
14474,dceb62e8fa94b46006c9554fed743df0,delivered,2017-08-01 18:23:30,2017-07-26 18:09:10,-6.009954
19268,5f9d46795c3126674e52becb3a1a517f,delivered,2017-07-20 23:03:42,2017-07-20 18:52:41,-0.174317
22520,b27af682321527a6349f1761eb3f360c,delivered,2017-06-27 14:51:54,2017-06-26 15:45:35,-0.962720
25393,1cc3ae63caffff2d6c3ee3e78e074acf,delivered,2017-08-10 18:28:56,2017-08-10 18:05:38,-0.016181
25646,e37f11cae9985ca58f0b56f268720537,delivered,2017-08-01 18:17:47,2017-07-31 17:49:56,-1.019340
27470,fa3e37584f4fdb1ded0e0de700dfcb4e,delivered,2017-08-09 18:18:43,2017-08-01 21:13:01,-7.878958
34939,c1e2bf2b7dd3309f2f5356c6b63968fa,delivered,2017-03-02 17:34:26,2017-02-14 15:15:57,-16.096169


In [83]:
# 8. 주문 상태별 분석 - 배송완료가 아닌데 배송일이 있는 경우
anomaly_8 = orders[
    (orders['order_status'] != 'delivered') & 
    (orders['order_delivered_customer_date'].notna())
]
anomalies['8. 미배송 상태인데 배송일 존재'] = anomaly_8

print("=" * 60)
print("8. 배송완료(delivered) 상태가 아닌데 고객 배송일이 존재")
print("   -> 상태와 배송일 불일치?!")
print("=" * 60)
print(f"이상 건수: {len(anomaly_8):,}건")
if len(anomaly_8) > 0:
    print("\n샘플 데이터:")
    display(anomaly_8[['order_id', 'order_status', 'order_delivered_carrier_date',
                       'order_delivered_customer_date']].head(10))


8. 배송완료(delivered) 상태가 아닌데 고객 배송일이 존재
   -> 상태와 배송일 불일치?!
이상 건수: 1건

샘플 데이터:


,order_id,order_status,order_delivered_carrier_date,order_delivered_customer_date
2921,1950d777989f6a877539f53795b4c3c3,canceled,2018-02-20 19:57:13,2018-03-21 22:03:51


In [84]:
# 이상 데이터 종합 요약

print("=" * 70)
print("배송 관련 이상 데이터 종합 요약")
print("=" * 70)
print(f"{'유형':<45} {'건수':>10} {'비율':>10}")
print("-" * 70)

total_orders = len(orders)
for name, df in anomalies.items():
    count = len(df)
    ratio = count / total_orders * 100
    print(f"{name:<45} {count:>10,}건")

배송 관련 이상 데이터 종합 요약
유형                                                    건수         비율
----------------------------------------------------------------------
1. 운송업체 인수일 > 고객 배송일                                  19건
2. 주문 승인일 > 운송업체 인수일                               1,359건
3. 주문일 > 주문 승인일                                        0건
4. 주문일 > 고객 배송일                                        0건
5. 배송완료 상태인데 배송일 없음                                    8건
6. 예상 배송일 < 주문일                                        0건
7. 배송 시간 0일 이하                                        19건
8. 미배송 상태인데 배송일 존재                                     1건


In [85]:
# 중복 제거
all_anomaly_ids = set()
for name, df in anomalies.items():
    all_anomaly_ids.update(df['order_id'].tolist())

print(f"총 이상 데이터 고유 주문 수: {len(all_anomaly_ids):,}건")
print(f"   (전체 주문의 {len(all_anomaly_ids)/total_orders*100:.2f}%)")

# 이상 데이터 DataFrame
anomaly_orders = orders[orders['order_id'].isin(all_anomaly_ids)].copy()

총 이상 데이터 고유 주문 수: 1,387건
   (전체 주문의 1.40%)


In [86]:
# ==========================================
# 판매자별 판매 건수 분석
# ==========================================

# 데이터 로드
order_items = pd.read_csv('Olist_DataSet/olist_order_items_dataset.csv')
sellers = pd.read_csv('Olist_DataSet/olist_sellers_dataset.csv')

print(f"주문 아이템 수 (필터링 전): {len(order_items):,}건")

# 필터링된 orders의 order_id만 사용
valid_order_ids = orders['order_id'].unique()
order_items = order_items[order_items['order_id'].isin(valid_order_ids)].copy()

print(f"주문 아이템 수 (필터링 후): {len(order_items):,}건")
print(f"전체 판매자 수: {len(sellers):,}명")

주문 아이템 수 (필터링 전): 112,650건
주문 아이템 수 (필터링 후): 112,279건
전체 판매자 수: 3,095명


In [87]:
# 판매자별 판매 건수 집계
seller_sales_count = order_items.groupby('seller_id').agg(
    판매건수=('order_id', 'count'),
    고유주문수=('order_id', 'nunique'),
    총매출=('price', 'sum'),
    평균단가=('price', 'mean')
).reset_index()

In [88]:
# 판매 건수 통계
print("=" * 60)
print("판매 건수 기술통계")
print("=" * 60)
print(seller_sales_count['판매건수'].describe())

print("\n" + "=" * 60)
print("판매 건수 구간별 판매자 수")
print("=" * 60)

# 판매 건수 구간 분류
bins = [0, 1, 5, 10, 20, 50, 100, 500, float('inf')]
labels = ['1건', '2~5건', '6~10건', '11~20건', '21~50건', '51~100건', '101~500건', '500건 초과']

seller_sales_count['판매건수_구간'] = pd.cut(seller_sales_count['판매건수'], bins=bins, labels=labels)

sales_distribution = seller_sales_count['판매건수_구간'].value_counts().sort_index()
sales_distribution_pct = (sales_distribution / len(seller_sales_count) * 100).round(2)

distribution_df = pd.DataFrame({
    '판매자 수': sales_distribution,
    '비율(%)': sales_distribution_pct
})
display(distribution_df)

판매 건수 기술통계
count    3068.000000
mean       36.596806
std       119.512228
min         1.000000
25%         2.000000
50%         8.000000
75%        25.000000
max      2033.000000
Name: 판매건수, dtype: float64

판매 건수 구간별 판매자 수


,판매자 수,비율(%)
판매건수_구간,,
1건,493,16.07
2~5건,832,27.12
6~10건,439,14.31
11~20건,436,14.21
21~50건,408,13.30
51~100건,224,7.30
101~500건,207,6.75
500건 초과,29,0.95
